## Main Code

### 0. Libraries

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00


In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
import json
import random
import time
import re
from datetime import datetime, timedelta
from faker import Faker
from typing import Tuple, List, Dict, Any

/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.cloud.resourcemanager_v3 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.resourcemanager_v3 past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarn

### 1. Configuration & Naming Standards

In [ ]:
# ==========================================
# 1. CONFIGURATION & NAMING STANDARDS
# ==========================================

PROJECT_ID = "project-nirvana-405904"  # <--- REPLACE THIS
LOCATION = "us-central1"

PROJECT_TAG = "csv"
SCRIPT_VERSION = "005"
TABLE_VERSION = "005"

JSONL_FILE = f"vel_{PROJECT_TAG}_transcripts_{SCRIPT_VERSION}.jsonl"
DATASET_ID = f"vel_{PROJECT_TAG}_schema"
TABLE_ID = f"vel_{PROJECT_TAG}_synthetic_transcripts_{TABLE_VERSION}"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-2.5-flash")

# Initialize Faker
fake = Faker()
Faker.seed(42)
random.seed(42)

### 2. Data Definitions (Personas, Products, Chaos)

In [ ]:
# ==========================================
# 2. DATA DEFINITIONS (Personas, Products, Chaos)
# ==========================================

PERSONAS = {
    "Legacy Enterprise": {
        "description": "Established 7+ years ago. High subs (2M-10M) but declining views.",
        "pain_points": ["Subscribers not getting notifications", "Fear of irrelevance"],
        "metrics": {"subs_range": (2000000, 10000000), "trend": "Declining"},
        "spm_goal": "Strategic Pivot (modernization, community posts)",
        "tone": "Professional, concerned, nostalgic"
    },
    "Short-Form Specialist": {
        "description": "Rapid growth via Shorts (500k-5M subs). High views, low revenue.",
        "pain_points": ["Monetization gap (low RPM)", "Brand safety"],
        "metrics": {"subs_range": (500000, 5000000), "trend": "Explosive"},
        "spm_goal": "Diversification (long-form, sponsorships)",
        "tone": "Energetic, impatient, volume-focused"
    },
    "Niche Professional": {
        "description": "High-value verticals (Finance, Tech). Lower views but high RPM.",
        "pain_points": ["Limited Ads (Yellow Icon)", "Feature access"],
        "metrics": {"subs_range": (100000, 500000), "trend": "Steady"},
        "spm_goal": "Product Adoption (Shopping, Memberships)",
        "tone": "Transactional, data-driven, calm"
    },
    "Viral Newcomer": {
        "description": "Sudden massive spike. Lacks infrastructure. Overwhelmed.",
        "pain_points": ["Identity Verification", "Copyright Claims"],
        "metrics": {"subs_range": (10000, 200000), "trend": "Viral"},
        "spm_goal": "Operational Health (Policy basics, Security)",
        "tone": "Excited, chaotic, anxious, informal"
    }
}

PRODUCT_ECOSYSTEM = {
    "Monetization": ["AdSense RPM vs CPM", "Channel Memberships setup", "Super Chat strategies", "YouTube Shopping tagging", "Shorts Ad Revenue Sharing model", "YouTube BrandConnect campaigns", "Super Thanks activation & engagement", "YouTube Premium revenue calculation"],
    "Content Formats": ["Shorts frequency (3x/week)", "Vertical Live discovery", "Podcast playlist setup", "Multi-Language Audio (Dubbing)", "YouTube Premieres (Trailers & Live Redirect)", "Community Tab strategy (Polls & Quizzes)", "Shorts Collab & Green Screen features", "Video Chapters (Manual vs. Automatic)"],
    "Tools & Policy": ["Copyright Match Tool", "Yellow Icon (Limited Ads) appeals", "Audience Retention graph analysis", "Ad-friendly guidelines Self-Certification", "Content ID claims vs. Copyright Strikes", "Community Guidelines warning resolution training", "YouTube Analytics Research Tab (Search Insights)"],
    "Events & Milestones": ["Gold Play Button delivery", "Fan Meetup security", "YouTube Creator Summits invitations", "YouTube FanFest participation", "Diamond Creator Award (10M) verification"]
}

SCENARIOS = [
    {"type": "Quarterly Business Review", "duration_minutes": 60, "focus": "Strategic planning", "chaos_probability": 0.2},
    {"type": "Annual Strategy Workshop", "duration_minutes": 90, "focus": "Long-term growth & multiple product adoption", "chaos_probability": 0.1},
    {"type": "Crisis & Channel Audit", "duration_minutes": 45, "focus": "Urgent issue resolution + Health Check", "chaos_probability": 0.6},
    {"type": "Content Strategy Deep Dive", "duration_minutes": 120, "focus": "Comprehensive content overhaul", "chaos_probability": 0.3}
]

CHAOS_INSTRUCTIONS = {
    "Technical Glitch": "The Creator's audio crackles. They have to repeat themselves. The SPM sounds slightly annoyed but hides it.",
    "Interruption": "A loud background noise (dog, doorbell) forces a 2-turn pause. The conversation loses its momentum.",
    "Jargon Misunderstanding": "The Creator thinks 'RPM' is 'Revenue Per Million' and gets excited; the SPM must correct the math.",
    "Tangent": "The Creator spends 3 turns complaining about a recent movie or the weather. The SPM has to struggle to bring the conversation back to business.",
    "Budget Stress": "The Creator focuses obsessively on 'how much this costs' and ignores the security/policy benefits for several turns.",
    "None": "Standard professional flow with minimal verbal fillers."
}

SPM_NAMES = [fake.name() for _ in range(20)]
NICHES = ["Gaming", "Beauty", "Tech Review", "Finance", "Vlog/Lifestyle", "Cooking"]

CHANNEL_PREFIXES = {
    "Gaming": ["Pixel", "Neon", "Retro", "Speed", "Shadow", "Elite", "Pro", "Quest", "Cyber", "Glitch"],
    "Beauty": ["Glow", "Pure", "Luxe", "Velvet", "Chic", "Radiant", "Bella", "Silk", "Trend", "Glam"],
    "Tech Review": ["Future", "Smart", "Tech", "Binary", "Gadget", "Silicon", "Digital", "Byte", "Hardware", "Logic"],
    "Finance": ["Wealth", "Market", "Crypto", "Asset", "Value", "Capital", "Bull", "Fiscal", "Invest", "Money"],
    "Vlog/Lifestyle": ["Daily", "Urban", "Wild", "Simple", "Happy", "Travel", "LifeWith", "Vibe", "Core", "Just"],
    "Cooking": ["Tasty", "Chef", "Golden", "Spicy", "Fresh", "Yummy", "Kitchen", "Baked", "Savory", "Sweet"]
}

CHANNEL_SUFFIXES = {
    "Gaming": ["Plays", "Gaming", "Arcade", "Zone", "Quest", "TV", "Live", "Station", "Hub", "Verse"],
    "Beauty": ["Beauty", "Cosmetics", "Skin", "Style", "Makeup", "Looks", "Secrets", "Studio", "Room", "Diaries"],
    "Tech Review": ["Reviews", "Lab", "Unboxed", "Flow", "Hub", "Central", "Insights", "Talks", "Breakdown", "Zone"],
    "Finance": ["Watch", "Flow", "Capital", "Sense", "Moves", "Tips", "Guru", "Strategies", "Roadmap", "Hustle"],
    "Vlog/Lifestyle": ["Vlogs", "Life", "Journeys", "Adventures", "Stories", "Days", "Moments", "World", "Lens", "Focus"],
    "Cooking": ["Kitchen", "Eats", "Bites", "Table", "Recipes", "Bakery", "Cooks", "Flavors", "Delights", "Spot"]
}

REGIONS = {
    "North America (USA)": {
        "code": "en-US",
        "style": "Direct, energetic, uses American idioms (bucks, awesome, dude).",
        "cultural_context": "Western business casual."
    },
    "Europe (UK)": {
        "code": "en-GB",
        "style": "Polite, perhaps slightly dry humor, uses British terms (cheers, mate, brilliant).",
        "cultural_context": "European formality mixed with wit."
    },
    "Asia Pacific (India)": {
        "code": "en-IN",
        "style": "Respectful, expressive, formal but warm, uses specific Indian English phrasing.",
        "cultural_context": "High deference to authority/policy."
    },
    "Latin America (Brazil)": {
        "code": "en-BR",
        "style": "Warm, engaging, enthusiastic, speaks English with a slight Portuguese cadence or phrasing.",
        "cultural_context": "Relationship-focused."
    },
    "Europe (Germany)": {
        "code": "en-DE",
        "style": "Direct, precise, efficient, less small talk, focuses on facts.",
        "cultural_context": "Efficiency-focused."
    }
}

### 3. Helper Functions

In [ ]:
# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================

def parse_raw_transcript_to_json(raw_text: str, start_time: datetime, expected_duration_minutes: int = 15) -> Tuple[List[Dict[str, Any]], str]:
    """
    Parse JSON array produced by the generator into the structured transcript.
    Includes DEFENSIVE LOGIC to handle 'Split-Object' hallucinations and 'Double-Escaping'.
    """
    structured_transcript: List[Dict[str, Any]] = []
    dialogue_list = []

    try:
        # 1. Cleaning Step: Handle Double Escaping (CSV style ""key"")
        clean_text = raw_text
        clean_text = re.sub(r"^```(?:json)?\s*", "", clean_text)
        clean_text = re.sub(r"\s*```$", "", clean_text)

        # Fix double quotes on keys if present (e.g. ""role"" -> "role")
        # Be careful not to break content that legitimately has quotes.
        # This regex targets double-quoted keys specifically.
        clean_text = re.sub(r'""(role|content)""', r'"\1"', clean_text)

        dialogue_list_raw = json.loads(clean_text)

        if not isinstance(dialogue_list_raw, list):
            raise ValueError("Parsed JSON is not an array.")

        # 2. Repair Logic: Handle Split-Object Hallucination
        # Symptom: [{"role": "role", "content": "SPM"}, {"role": "content", "content": "text..."}]

        current_speaker = None

        for item in dialogue_list_raw:
            # Check if this item is a valid, complete turn
            if "role" in item and "content" in item:

                # CASE A: Standard Valid Item
                # If role is SPM or Creator, and content looks like a message
                if item["role"] in ["SPM", "Creator", "Action"] and item["role"] != "role":
                    dialogue_list.append(item)
                    current_speaker = None # Reset
                    continue

                # CASE B: The "Split-Object" Defect
                # Sub-case B1: The item defines the ROLE (e.g. role="role", content="SPM")
                if item["role"] == "role" or item.get("content") in ["SPM", "Creator"]:
                    # We found the speaker, store it for the next iteration
                    current_speaker = item.get("content")
                    if current_speaker not in ["SPM", "Creator", "Action"]:
                        current_speaker = None # Invalid speaker value

                # Sub-case B2: The item defines the CONTENT (e.g. role="content", content="Hello")
                elif item["role"] == "content" and current_speaker:
                    # We have a speaker waiting from previous loop, and now we have content
                    dialogue_list.append({
                        "role": current_speaker,
                        "content": item["content"]
                    })
                    current_speaker = None # Consumed

    except Exception as e:
        print(f"⚠️ JSON Parsing Error (Fallback active): {e}")
        # Fallback: Regex extraction
        dialogue_list = []
        for line in raw_text.splitlines():
            m = re.match(r'^\s*(SPM|Creator)\s*[:\-]\s*(.+)', line)
            if m:
                dialogue_list.append({"role": m.group(1), "content": m.group(2).strip()})
        if not dialogue_list:
            return [], start_time.isoformat()

    # 3. Standard Processing (Timestamping)
    # If repair resulted in empty list, return empty
    if not dialogue_list:
         return [], start_time.isoformat()

    total_msgs = max(1, len(dialogue_list))
    total_seconds = max(1, int(expected_duration_minutes * 60))

    for idx, entry in enumerate(dialogue_list):
        role = entry.get("role")
        content = entry.get("content", "")

        # Normalize speaker code for BigQuery/Downstream
        if role == "SPM":
            speaker_code = "A"
        elif role == "Creator":
            speaker_code = "B"
        else:
            speaker_code = "Action"

        frac = idx / (total_msgs - 1) if total_msgs > 1 else 0
        base_seconds = int(frac * total_seconds)
        jitter = random.randint(-5, 10)
        msg_time = start_time + timedelta(seconds=max(0, base_seconds + jitter))

        structured_transcript.append({
            "speaker": speaker_code,
            "timestamp": msg_time.isoformat(),
            "text": content
        })

    end_time = start_time + timedelta(seconds=total_seconds + random.randint(0, 10))
    return structured_transcript, end_time.isoformat()


def normalize_record_for_jsonl(rec: Dict[str, Any]) -> Dict[str, Any]:
    """
    Clean up types for BigQuery ingestion.
    Simplified to remove validation specific fields.
    """
    normalized = rec.copy()

    for k in ("recording_start", "recording_end"):
        v = normalized.get(k)
        if isinstance(v, datetime):
            normalized[k] = v.isoformat()
        else:
            try:
                datetime.fromisoformat(v)
                normalized[k] = v
            except Exception:
                normalized[k] = ""

    try:
        normalized["duration_minutes"] = float(normalized.get("duration_minutes", 0.0))
    except Exception:
        normalized["duration_minutes"] = 0.0

    return normalized

### 4. Generation Engine

In [ ]:
# ==========================================
# 4. GENERATION ENGINE
# ==========================================

class CreatorAgent:
    def __init__(self, id: int, persona_name: str):
        self.id = id
        self.persona_name = persona_name
        self.persona_data = PERSONAS[persona_name]
        self.niche = random.choice(NICHES)
        self.experience_level = self.persona_data.get("experience_level", "intermediate")
        self.subs = random.randint(self.persona_data['metrics']['subs_range'][0], self.persona_data['metrics']['subs_range'][1])

        self.region_name = random.choice(list(REGIONS.keys()))
        self.region_data = REGIONS[self.region_name]

        prefix = random.choice(CHANNEL_PREFIXES[self.niche])
        suffix = random.choice(CHANNEL_SUFFIXES[self.niche])
        unique_tail = str(random.randint(1, 999)) if random.random() < 0.6 else fake.word().capitalize()
        self.channel_name = f"{prefix}{suffix}{unique_tail}"

        self.creator_display_name = fake.name()
        self.assigned_spm = random.choice(SPM_NAMES) # We don't use faker on this to have a SPM with several creators
        self.history: List[str] = []

class SimulationEngine:
    def __init__(self, target_rows=10):
        self.target_rows = target_rows
        self.transcript_db = []

    # HELPER: Flatten topics for easy selection
    def get_all_topics(self):
        all_topics = []
        for category, topics in PRODUCT_ECOSYSTEM.items():
            all_topics.extend(topics)
        return all_topics

    def generate_system_prompt(self, creator, scenario, chaos_key, topic_list, is_followup):
        history_context = f"PREVIOUS CONTEXT: {creator.history[-1]}" if is_followup and creator.history else "This is the first call."
        chaos_instruction = CHAOS_INSTRUCTIONS.get(chaos_key, "No interruptions.")
        duration = scenario.get('duration_minutes', 60)

        # Calculate turns based on duration (approx 1.2 turns per minute for long calls to stay within token limits)
        target_turns = int(duration * 1.2)

        # Format the list of topics into a bulleted string
        agenda_items = "\n".join([f"   - {t}" for t in topic_list])

        # Dynamic Structure Guide based on DURATION
        if duration >= 90: # Very Long Call (1.5 - 2 hours)
            structure_guide = f"""
            **MANDATORY STRUCTURE (DEEP DIVE CALL - {duration} mins):**
            1. [0-10 min] Warm-up, relationship building, and catching up on personal life/culture.
            2. [10-20 min] Channel Health Check: Reviewing analytics (Views, RPM, Subscribers).
            3. [20-80 min] **THE AGENDA (Core Discussion):** Discuss the following topics sequentially. Allow natural transitions between them:
            {agenda_items}
            4. [80-100 min] Brainstorming session for future content ideas based on these topics.
            5. [100-{duration} min] Defining clear action items and closing.
            """
        elif duration >= 45: # Standard QBR (45-60 mins)
            structure_guide = f"""
            **MANDATORY STRUCTURE (STRATEGY CALL - {duration} mins):**
            1. [0-5 min] Intro and check-in.
            2. [5-15 min] Performance Review.
            3. [15-{duration-10} min] **Main Topics:** Cover these points in detail:
            {agenda_items}
            4. [{duration-10}-{duration} min] Q&A and Closing.
            """
        else: # Short Call (Crisis)
            structure_guide = f"""
            **MANDATORY STRUCTURE (SHORT/CRISIS CALL - {duration} mins):**
            1. [0-5 min] Immediate Triage: Address the main crisis ({topic_list[0]}).
            2. [5-{duration-5} min] Brief check on secondary topics if time permits:
            {agenda_items}
            3. [{duration-5}-{duration} min] Wrap up.
            """

        realism_guidelines = """
        HUMAN REALISM GUIDELINES:
        - Add false starts, partial overlaps, self-interruptions.
        - Include jargon misunderstanding.
        - Allow topic drift.
        - Make it feel recorded live, not scripted.
        """

        # BLOCK: JSON INTEGRITY RULES
        # Explicitly forbids the split-object hallucination observed in the incident report.
        json_integrity_rules = """
        **CRITICAL JSON FORMATTING RULES (STRICT ENFORCEMENT):**
        1. **NO SPLIT OBJECTS:** Do NOT separate the role and the content into different objects.
           - ❌ INCORRECT: [{"role": "role", "content": "SPM"}, {"role": "content", "content": "Hello"}]
           - ✅ CORRECT:   [{"role": "SPM", "content": "Hello"}]
        2. **NO DOUBLE QUOTES ON KEYS:** Do not use CSV-style double escaping.
           - ❌ INCORRECT: [{""role"": ""SPM""}]
           - ✅ CORRECT:   [{"role": "SPM"}]
        3. **SINGLE OBJECT PER TURN:** Each list item must contain BOTH "role" and "content".
        """

        return f"""
        Generate a realistic, verbatim call transcript between a YouTube SPM and a Creator.

        {realism_guidelines}

        **ROLE 1: SPM ({creator.assigned_spm})**
        - Goal: Cover the full agenda of {len(topic_list)} topics while maintaining the relationship.
        - Tone: Professional, uses jargon (YPP, CTR), empathetic.

        **ROLE 2: CREATOR ({creator.channel_name})**
        - Persona: {creator.persona_name} ({creator.persona_data['tone']}).
        - Region: {creator.region_name} ({creator.region_data['style']})
        - Pain Point: {creator.persona_data['pain_points'][0]}.

        **SCENARIO:** {scenario['type']} ({duration} minutes).
        **CHAOS:** {chaos_instruction}
        **CONTEXT:** {history_context}

        **TOPICS TO DISCUSS (AGENDA):**
        {agenda_items}

        {structure_guide}

        {json_integrity_rules}

        **INSTRUCTIONS:**
        1. **LENGTH:** At least **{target_turns} dialogue turns**. DO NOT SUMMARIZE.
        2. **DEPTH:** You must touch upon ALL topics listed in the Agenda.
        3. **FORMAT:** Output a VALID JSON ARRAY:
           [
             {{"role": "SPM", "content": "..."}},
             {{"role": "Creator", "content": "..."}}
           ]
        """

    def run(self):
        print(f"🚀 Starting generation. Target: {self.target_rows} rows...")

        rows_generated = 0
        creator_id_counter = 1
        all_possible_topics = self.get_all_topics() # Get flat list of topics

        while rows_generated < self.target_rows:
            creator = CreatorAgent(creator_id_counter, random.choice(list(PERSONAS.keys())))
            creator_id_counter += 1
            num_calls = random.choices([1, 2, 3, 4], weights=[50, 30, 15, 5], k=1)[0]

            print(f"Processing {creator.channel_name} ({creator.region_name}) - Planning {num_calls} call(s)...")

            for call_idx in range(num_calls):
                if rows_generated >= self.target_rows: break

                scenario = random.choice(SCENARIOS)
                duration = scenario.get('duration_minutes', 60)
                chaos_key = "None" if random.random() > scenario['chaos_probability'] else random.choice(list(CHAOS_INSTRUCTIONS.keys()))

                # --- TOPIC SELECTION LOGIC (NEW) ---
                # Determine how many topics based on duration
                if duration >= 90:
                    num_topics = random.randint(10, 18) # Deep dive: 8 to 15 topics
                elif duration >= 45:
                    num_topics = random.randint(6, 10)  # Standard: 4 to 8 topics
                else:
                    num_topics = random.randint(3, 5)  # Crisis: 1 to 3 topics

                # Select random unique topics
                selected_topics = random.sample(all_possible_topics, min(num_topics, len(all_possible_topics)))

                # Primary topic for metadata (the first one)
                primary_topic = selected_topics[0]
                # Join all topics for metadata storage (optional, or just store list)
                full_topic_string = "; ".join(selected_topics)

                start_dt = datetime.now() - timedelta(days=random.randint(1, 30))
                conv_id = f"{creator.id}_{call_idx}_{int(time.time())}"

                try:
                    # 1. Generate Raw Transcript with MULTIPLE TOPICS
                    prompt = self.generate_system_prompt(creator, scenario, chaos_key, selected_topics, call_idx > 0)

                    response = model.generate_content(
                        prompt,
                        generation_config={
                            "temperature": 0.7,
                            "response_mime_type": "application/json",
                            "max_output_tokens": 8192 # Crucial for long calls
                        }
                    )

                    try:
                        raw_transcript_json_string = response.text
                    except ValueError:
                        parts = response.candidates[0].content.parts
                        raw_transcript_json_string = "".join([part.text for part in parts])

                    # 2. Parse
                    structured_transcript, real_end_time_iso = parse_raw_transcript_to_json(
                        raw_transcript_json_string,
                        start_dt,
                        expected_duration_minutes=duration
                    )

                    # 3. Summarize
                    summary = model.generate_content(f"Summarize in 1 sentence: {raw_transcript_json_string[:2000]}").text.strip()
                    creator.history.append(summary)

                    # 4. Calculate duration
                    try:
                        start_iso = start_dt
                        end_iso = datetime.fromisoformat(real_end_time_iso)
                        duration_minutes_real = round((end_iso - start_iso).total_seconds() / 60.0, 2)
                    except Exception:
                        duration_minutes_real = float(duration)

                    # 5. Build Record
                    record = {
                        "conversation_id": conv_id,
                        "creator_id": creator.id,
                        "channel_name": creator.channel_name,
                        "creator_niche": creator.niche,
                        "creator_persona": creator.persona_name,
                        "creator_tone": creator.persona_data['tone'],
                        "spm_name": creator.assigned_spm,
                        "creator_region": creator.region_name,
                        "language_code": creator.region_data['code'],
                        "scenario": scenario['type'],
                        "product_topic": full_topic_string, # Store ALL topics here
                        "duration_minutes": duration_minutes_real,
                        "recording_start": start_dt.isoformat(),
                        "recording_end": real_end_time_iso,
                        "raw_transcript": raw_transcript_json_string
                    }

                    self.transcript_db.append(record)
                    rows_generated += 1
                    print(f"  ✅ Generated: {conv_id} ({len(selected_topics)} topics, ~{duration_minutes_real} mins)")

                    time.sleep(1)

                except Exception as e:
                    print(f"Error processing call {conv_id}: {e}")

        # Final Save
        with open(JSONL_FILE, 'w', encoding='utf-8') as f:
            for entry in self.transcript_db:
                clean_entry = normalize_record_for_jsonl(entry)
                f.write(json.dumps(clean_entry, ensure_ascii=False, separators=(',', ':')) + '\n')
        print(f"✅ Saved {rows_generated} transcripts to: {JSONL_FILE}")

### 5. BigQuery Upload

In [ ]:
# ==========================================
# 5. BIGQUERY UPLOAD
# ==========================================

def upload_to_bigquery():
    client = bigquery.Client(project=PROJECT_ID)
    dataset_ref = client.dataset(DATASET_ID)

    try:
        client.get_dataset(dataset_ref)
    except NotFound:
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = "US"
        client.create_dataset(dataset)
        print(f"✅ Created dataset {DATASET_ID}")

    table_ref = dataset_ref.table(TABLE_ID)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.autodetect = True
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    with open(JSONL_FILE, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    job.result()
    print(f"🎉 Success! Loaded {job.output_rows} rows.")

## Run Main

In [ ]:
if __name__ == "__main__":
    start_time = time.perf_counter()

    # A. Create synthetic data
    sim = SimulationEngine(target_rows=20)
    sim.run()
    end_time_sd = time.perf_counter()

    # B. Upload to BigQuery
    upload_to_bigquery()

    end_time = time.perf_counter()
    duration_sd = str(timedelta(seconds=end_time_sd - start_time))
    duration_bq = str(timedelta(seconds=end_time - end_time_sd))
    duration_total = str(timedelta(seconds=end_time - start_time))

    print(f"Synthetic Data execution time (HH:MM:SS): {duration_sd}")
    print(f"Upload to BigQuery execution time (HH:MM:SS): {duration_bq}")
    print(f"Total execution time (HH:MM:SS): {duration_total}")

🚀 Starting generation. Target: 20 rows...
Processing SpeedZone105 (Asia Pacific (India)) - Planning 2 call(s)...
  ✅ Generated: 1_0_1771530949 (6 topics, ~60.0 mins)
  ✅ Generated: 1_1_1771530986 (7 topics, ~45.12 mins)
Processing WildWorldAgreement (Europe (UK)) - Planning 1 call(s)...
⚠️ JSON Parsing Error (Fallback active): Unterminated string starting at: line 367 column 13 (char 23816)
  ✅ Generated: 2_0_1771531023 (13 topics, ~0.0 mins)
Processing JustAdventures725 (Europe (UK)) - Planning 1 call(s)...
  ✅ Generated: 3_0_1771531075 (8 topics, ~60.05 mins)
Processing TrendBeautyTrue (Europe (Germany)) - Planning 1 call(s)...
  ✅ Generated: 4_0_1771531118 (13 topics, ~120.0 mins)
Processing RadiantMakeupGrow (Latin America (Brazil)) - Planning 1 call(s)...
⚠️ JSON Parsing Error (Fallback active): Unterminated string starting at: line 352 column 16 (char 25281)
  ✅ Generated: 5_0_1771531169 (13 topics, ~0.0 mins)
Processing LogicBreakdown922 (Latin America (Brazil)) - Planning 1 cal

In average it takes ~36 seconds to create a transcription